## Basic Exploration of the Daily Dataset

The objective of this notebook is to find problems in the data set of daily history and posible cleaup strategies before doing the full analysis.

### Reading the dataset

The data set is saved in a hierarchical maner having independent directories for year and subdirectories for each month.

In [3]:
import pandas as pd

currentTz = 'America/Merida'

# Changing the default timezone for the analysis
#spark.conf.set("spark.sql.session.timeZone", currentTz)

pd.date_range(start='2016-01',end='2016-12',freq='MS',tz=currentTz)



DatetimeIndex(['2016-01-01 00:00:00-06:00', '2016-02-01 00:00:00-06:00',
               '2016-03-01 00:00:00-06:00', '2016-04-01 00:00:00-06:00',
               '2016-05-01 00:00:00-05:00', '2016-06-01 00:00:00-05:00',
               '2016-07-01 00:00:00-05:00', '2016-08-01 00:00:00-05:00',
               '2016-09-01 00:00:00-05:00', '2016-10-01 00:00:00-05:00',
               '2016-11-01 00:00:00-06:00', '2016-12-01 00:00:00-06:00'],
              dtype='datetime64[ns, America/Merida]', freq='MS')